In [52]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [53]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

In [54]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [55]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 287
Component distribution: {'min': 1, 'p5': 1, 'max': 185, 'p999': 185, 'p99': 14, 'p1': 1, 'p10': 1, 'p90': 1, 'p50': 1, 'p25': 1, 'p75': 1, 'p95': 1, 'mean': 1.9407665505226481}


In [56]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                             95
nodePropertiesWritten                                                  557
ranLevels                                                                3
didConverge                                                           True
nodeCount                                                              557
communityCount                                                         296
communityDistribution    {'min': 1, 'p5': 1, 'max': 45, 'p999': 45, 'p9...
modularity                                                        0.718684
modularities             [0.6312269964407168, 0.7122485060806366, 0.718...
postProcessingMillis                                                    46
preProcessingMillis                                                      1
computeMillis                                                          142
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [57]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 1671}]

In [58]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [59]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,331,1.0,1.0,1.0,4.0,18.90,33
1,1,329,3.0,3.0,3.0,10.2,41.00,49
2,2,303,8.0,10.0,17.5,28.0,61.96,81


In [62]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | apoc.map.removeKeys(n{.*, type: [el in labels(n) WHERE el <> '__Entity__'][0]}, ['embedding', 'wcc', 'communities'])] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info[0]

{'communityId': '1-259',
 'nodes': [{'id': '單側疝氣',
   'description': '8%',
   'type': 'Medical_procedure',
   'sources': ['台灣人壽新住院醫療保險附約.pdf']},
  {'id': '精神疾病',
   'description': '因本人或其配偶患有礙優生之遺傳性、傳染性疾病或精神疾病',
   'type': 'Condition',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf']},
  {'id': '流產',
   'description': '因醫療行為所必要之流產',
   'type': 'Event',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '遺傳性疾病',
   'description': '因本人或其配偶患有礙優生之遺傳性、傳染性疾病或精神疾病',
   'type': 'Condition',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf']},
  {'id': '傳染性疾病',
   'description': '因本人或其配偶患有礙優生之遺傳性、傳染性疾病或精神疾病',
   'type': 'Condition',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf']},
  {'id': '四親等以內之血親',
   'description': '因本人或其配偶之四親等以內之血親患有礙優生之遺傳性疾病',
   'type': 'Person',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf']},
  {'id': '醫療行為所必要之流產', 'type': 'Event', 'sources': ['台灣人壽新住院醫療保險附約.pdf']},
  {'id': '本人或其配偶患有礙優生之遺傳性、傳染性疾病或精神疾病',
   'type': 'Event',
   'sources': ['台灣人壽新住院醫療保險附約.pdf']},
  {'id': '本人或其配偶之四親等以

In [46]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [11]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """
{community_info}
Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "請根據提供同社區的資訊包含 nodes 與 relationships, 產生同社區的自然語言的摘要資訊, 需要區分共同資訊以及各來源的資訊. No pre-amble.",
        ),
        (
            "system", "使用繁體中文回應"
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [48]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}, source: {node['source']},\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '1-259',
 'summary': '共同資訊:\n- 流產: 因醫療行為所必要之流產，出現在兩個來源文件中（台灣人壽龍實在住院醫療健康保險附約.pdf 和 台灣人壽新住院醫療保險附約.pdf）。\n- 遺傳性疾病、傳染性疾病、精神疾病: 因本人或其配偶患有礙優生之遺傳性、傳染性疾病或精神疾病，出現在台灣人壽龍實在住院醫療健康保險附約.pdf 中。\n\n不同來源的資訊:\n- 單側疝氣: 8%，僅出現在台灣人壽新住院醫療保險附約.pdf 中。\n- 四親等以內之血親: 因本人或其配偶之四親等以內之血親患有礙優生之遺傳性疾病，僅出現在台灣人壽龍實在住院醫療健康保險附約.pdf 中。\n- 醫療行為所必要之流產: 僅出現在台灣人壽新住院醫療保險附約.pdf 中。\n- 本人或其配偶患有礙優生之遺傳性、傳染性疾病或精神疾病: 僅出現在台灣人壽新住院醫療保險附約.pdf 中。\n- 本人或其配偶之四親等以內之血親患有礙優生之遺傳性疾病: 僅出現在台灣人壽新住院醫療保險附約.pdf 中。'}

In [14]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def process_community_with_llm(community_info, max_retry=0):
    '''
    params:
        community_info: [ { 
                            'communityId': str, 'nodes': [{'id': str, 'description': str|None, 'type': str}, ...], 
                            'rels': [{'start': str, 'description': str|None, 'type': str, 'end': 'str}, ...]
                          },
                          ... ]
        max_retry: 最多嘗試次數, 假設為2, 則最多遞迴執行 2+1=3次
    '''
    summaries = []
    faild_communities = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_community, community): community for community in community_info}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
            try:
                summaries.append(future.result())
            except Exception as e:
                community = futures[future]
                print(f'process community faild!:{community}, error:\n{e}')
                faild_communities.append(community)
    if len(faild_communities) > 0 and max_retry > 0:
        summaries.extend(process_community_with_llm(faild_communities, max_retry=max_retry-1))
    return summaries
summaries = process_community_with_llm(community_info, max_retry=0)

Processing communities: 100%|██████████| 77/77 [00:52<00:00,  1.48it/s]


In [15]:
summaries

[{'community': '2-168',
  'summary': '在台灣人壽的住院醫療保險附約中，主要涉及被保險人的各種情況和行為。被保險人可能因同一疾病或傷害引起併發症，並在同一醫院接受門診外科手術或附表二的特定處置。被保險人享有全民健康保險，並在申請時不需要提供健康證明文件。保險涵蓋的年齡上限為八十五歲。被保險人可能經歷出院，並參與門診外科手術。保險條款中還提到被保險人可能因心臟病、不孕症、人工受孕、美容手術、非法施用毒品等多種原因接受治療或手術。此外，保險條款還包括對被保險人的通知，以確保其權益。\n\n來源: 台灣人壽新住院醫療保險附約.pdf, 台灣人壽龍實在住院醫療健康保險附約.pdf'},
 {'community': '0-272',
  'summary': '在台灣人壽的住院醫療保險附約中，主要涉及被保險人的各種情況和行為。被保險人可能因同一疾病或傷害引起併發症，並在同一醫院接受門診外科手術或附表二的特定處置。被保險人享有全民健康保險，並在申請時不需要提供健康證明文件。保險涵蓋的年齡上限為八十五歲。被保險人可能經歷出院，並參與門診外科手術。保險條款中還提到被保險人可能因心臟病、不孕症、人工受孕、美容手術、非法施用毒品等多種原因接受治療或手術。此外，保險條款還包括對被保險人的通知，以確保其權益。\n\n來源: 台灣人壽新住院醫療保險附約.pdf, 台灣人壽龍實在住院醫療健康保險附約.pdf'},
 {'community': '1-1',
  'summary': '《台灣人壽新住院醫療保險附約》與《台灣人壽龍實在住院醫療健康保險附約》詳細說明了保險契約的各項條款與規定。保險責任始於要保人交付第一期保險費後，並由本公司發給保險單作為承保憑證。要保人需履行告知義務，並在住址變更時通知本公司。附約效力會因主契約效力停止而同時停止，並且有寬限期間的規定。續保時需重新計算保險費，最高續保年齡為二十三歲。要保人有權在保險單送達後十日內撤銷附約，並可申請附約復效。保險事故發生後，本公司負有給付保險金的責任，但若要保人未繳清保險費，保險金將扣除欠繳部分。附約的終止與解除權利也有詳細規定。\n\n來源: 台灣人壽新住院醫療保險附約.pdf, 台灣人壽龍實在住院醫療健康保險附約.pdf'},
 {'community': '0-315',
  'summa

In [16]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]

In [ ]:
graph.query('''
MATCH (n:__Entity__)
WHERE n.description IS NOT NULL and apoc.meta.cypher.type(n.description) <> 'STRING'
REMOVE n.embedding
SET n.description = apoc.text.join(n.description, "\n---\n")
RETURN n.description
''')

[{'n.description': '1,000, 1,500'},
 {'n.description': '90,000, 120,000'},
 {'n.description': '160,000, 180,000'},
 {'n.description': '40,000, 60,000'},
 {'n.description': '600, 900'},
 {'n.description': '2,000, 3,000'},
 {'n.description': '50%, 32%'}]

In [18]:
graph.query(
    """
MATCH (n:`__Community__`)<-[:IN_COMMUNITY]-()<-[:HAS_ENTITY]-(c)
WITH n, count(distinct c) AS chunkCount
SET n.weight = chunkCount"""
)

[]